In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 

In [2]:
import json
with open(r"train.json", "r") as read_file:
  train = json.load(read_file)

In [ ]:
train

[{'context': "module icons { \n yang-version 1.1; \n namespace 'http://anutanetworks.com/icons'; \n prefix ico; \n\n organization \n 'Anuta Networks'; \n \n revision 2021-06-30 { \n description \n 'Report Infra Icons'; \n } \n\n container icons { \n list category { \n key 'name'; \n leaf name { \n type string; \n } \n leaf description { \n type string; \n } \n list icon { \n key 'name'; \n leaf name { \n type string; \n } \n leaf description { \n type string; \n } \n leaf url { \n type string; \n } \n } \n } \n } \n } \n. Based on the above YANG model return me RESTCONF-API method and request body for the questions asked, [Method: GET; API: /restconf/data/icons:icons/category]",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Retrieve all the available icons?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/icons:icons/category',
      'answer_start': 580}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'Give me all the icons?',
    

In [3]:
import json
with open(r"test.json", "r") as read_file:
  test = json.load(read_file)

In [ ]:
test

[{'context': "module alert-group { \n namespace 'http://example.com/alert-group'; \n prefix alert-group; \n  \ncontainer alert-groups { \n container config { \n leaf-list name { \n type string; \n } \n } \n list group-config { \n key 'tenant'; \n leaf tenant { \n type string; \n } \n leaf-list name { \n type string; \n } \n } \n } \n }. Based on the above YANG model return me RESTCONF-API method and request body for the questions asked, [Method: GET; API: /restconf/data/alert-group:alert-groups]",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Retrieve all the alert groups?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
      'answer_start': 432}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'Can you give me all the alert groups?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
      'answer_start': 432}]}]}]

In [4]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
import torch
torch.cuda.empty_cache()

In [5]:
model_type = "bert"
model_name = "bert-base-uncased"

In [6]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    confl

In [ ]:
args = QuestionAnsweringArgs()


In [7]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 20,
    "evaluate_during_training_steps": 1000,
    # "wandb_project": "Question Answer Application",
    # "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    "train_batch_size": 128,
    "eval_batch_size": 64,
}

In [ ]:
!rm -rf outputs

In [8]:
model = QuestionAnsweringModel(
    model_type, model_name, args=train_args
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [9]:
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 5/5 [00:00<00:00, 57143.11it/s]


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "

convert squad examples to features: 100%|██████████| 2/2 [00:00<00:00, 167.13it/s]

add example index and unique id: 100%|██████████| 2/2 [00:00<00:00, 23899.17it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 18 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(20,
 {'global_step': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20],
  'correct': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'similar': [0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  'incorrect': [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'train_loss': [4.753125190734863,
   4.76953125,
   4.714062690734863,
   3.9865236282348633,
   3.196484327316284,
   2.5601563453674316,
   1.802343726158142,
   1.300146460533142,
   0.9734863042831421,
   0.672436535358429,
   0.48029783368110657,
   0.3706299066543579,
   0.25574952363967896,
   0.17088621854782104,
   0.16478270292282104,
   0.11592407524585724,
   0.10980834811925888,
   0.092376708984375,
   0.07353515923023224,
   0.068695068359375],
  'eval_loss': [0.282958984375,
   0.282958984375,
   0.01451873779296875,
   -0.2442626953125,
   -0.5234375,
   -0.80810546875,
   -1.0859375,
   -1.3486

In [10]:
results, text = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
results

{'correct': 0, 'similar': 2, 'incorrect': 0, 'eval_loss': -3.1171875}

In [12]:
text

{'correct_text': {},
 'similar_text': {'00001': {'truth': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
   'predicted': '',
   'question': 'Retrieve all the alert groups?'},
  '00002': {'truth': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
   'predicted': '',
   'question': 'Can you give me all the alert groups?'}},
 'incorrect_text': {}}

In [13]:
import json
with open(r"predictions.json", "r") as read_file:
  to_predict = json.load(read_file)

In [ ]:
to_predict

[{'context': "module feature-toggle { \n namespace 'http://example.com/feature-toggle'; \n prefix ft; \n \n revision 2020-05-14 { \n description \n 'Initial revision'; \n } \n \n container feature-toggles { \n list feature-toggle { \n 0description \n 'The feature toggles enables a state information to decide if a feature/functionality should be enabled or disabled.'; \n key 'name'; \n leaf name { \n type string; \n description \n 'This is a predefined string value, usually populated by seed data only. This predefined key can be used to identify the features'; \n mandatory true; \n } \n leaf enable { \n type boolean; \n description \n 'This value tells whether to activate the feature'; \n } \n leaf description { \n type string; \n description \n 'Brief description about the feature toggle'; \n } \n } \n } \n }. Based on the above YANG model return me RESTCONF-API method and request body for the questions asked, [Method: GET; API: /restconf/data/feature-toggle:feature-toggles/feature-tog

In [18]:
final_answer, probablities = model.predict(to_predict)
print(final_answer)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 12748.64it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['empty']}]
